In [2]:
!pip install pytest-mock

In [3]:
!pip install pytest pytest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [11]:
%%writefile cash.py
import asyncio
import threading

class AsyncEventLoop:
    def __init__(self):
        self.loop = asyncio.new_event_loop()
        self.thread = threading.Thread(target=self.run_loop, daemon=True)
        self.thread.start()
        self._event = threading.Event()

    def run_loop(self):
        asyncio.set_event_loop(self.loop)
        self.loop.run_forever()

    def stop(self):
        self.loop.call_soon_threadsafe(self.loop.stop)

    def run_until_complete(self, coro):
        return asyncio.run_coroutine_threadsafe(coro, self.loop).result()

    async def wait(self):
        while not self._event.is_set():
            await asyncio.sleep(0.1)

    def set_event(self):
        self._event.set()

    def clear_event(self):
        self._event.clear()

# Mock class

class CurrencyConversion:
    def __init__(self, symbol: str, conversion_rate: float, event_manager: AsyncEventLoop):
        self.symbol = symbol
        self.conversion_rate = conversion_rate
        self.conversion_rate_updated = event_manager

    async def update(self):
        await asyncio.sleep(1)
        self.conversion_rate *= 1.01
        self.conversion_rate_updated.set_event()

# Main Cash Class
class Cash:
    def __init__(self, symbol: str, amount: float, conversion_rate: float, event_manager: AsyncEventLoop):
        if not symbol:
            raise ValueError("Symbol cannot be null or empty")
        self.symbol = symbol.upper()
        self.amount = amount
        self.currency_conversion = CurrencyConversion(self.symbol, conversion_rate, event_manager)
        self.lock = asyncio.Lock()
        self.updated_event = event_manager
        self.currency_conversion_updated_event = event_manager

    @property
    async def value_in_account_currency(self):
        return self.amount * self.currency_conversion.conversion_rate

    async def add_amount(self, amount: float) -> float:
        async with self.lock:
            self.amount += amount
            self.updated_event.set_event()
        return self.amount

    async def set_amount(self, amount: float):
        async with self.lock:
            if self.amount != amount:
                self.amount = amount
                self.updated_event.set_event()

    async def update_conversion_rate(self):
        await self.currency_conversion.update()
        self.currency_conversion_updated_event.set_event()

Overwriting cash.py


In [12]:
%%writefile test_cash.py
import pytest
from unittest.mock import AsyncMock
from cash import Cash, AsyncEventLoop

@pytest.fixture
def cash():
    event_loop = AsyncEventLoop()
    return Cash("USD", 1000, 1.0, event_loop)

@pytest.mark.asyncio
async def test_add_amount(cash):
    new_amount = await cash.add_amount(500)
    assert new_amount == 1500
    assert cash.amount == 1500

@pytest.mark.asyncio
async def test_set_amount(cash):
    await cash.set_amount(800)
    assert cash.amount == 800

@pytest.mark.asyncio
async def test_update_conversion_rate(cash, mocker):
    new_callable = AsyncMock()
    mocker.patch.object(cash.currency_conversion, 'update', new_callable)
    await cash.update_conversion_rate()
    cash.currency_conversion.conversion_rate = 1.05
    assert cash.currency_conversion.conversion_rate == 1.05

@pytest.mark.asyncio
async def test_value_in_account_currency(cash, mocker):
    new_callable = AsyncMock()
    mocker.patch.object(cash.currency_conversion, 'update', new_callable)
    cash.currency_conversion.conversion_rate = 1.05
    await cash.update_conversion_rate()
    value = await cash.value_in_account_currency
    assert value == cash.amount * cash.currency_conversion.conversion_rate

@pytest.mark.asyncio
async def test_update_conversion_rate_multiple(cash, mocker):
    new_callable = AsyncMock()
    mocker.patch.object(cash.currency_conversion, 'update', new_callable)
    cash.currency_conversion.conversion_rate = 1.0
    await cash.update_conversion_rate()
    cash.currency_conversion.conversion_rate = 1.10

    assert cash.currency_conversion.conversion_rate == 1.10

@pytest.mark.asyncio
async def test_value_in_account_currency_multiple(cash, mocker):
    new_callable = AsyncMock()
    mocker.patch.object(cash.currency_conversion, 'update', new_callable)
    cash.currency_conversion.conversion_rate = 1.10
    await cash.update_conversion_rate()
    value = await cash.value_in_account_currency

    assert value == cash.amount * cash.currency_conversion.conversion_rate


Overwriting test_cash.py


In [13]:
!pytest test_cash.py --asyncio-mode=auto

/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 6 items                                                                   